# Titulo

In [1]:
import pandas as pd
import os
import time 
from tinydb import TinyDB, Query
from pytubefix import YouTube
from pytubefix.exceptions import VideoUnavailable
from pytubefix.cli import on_progress
import whisper
from whisper.utils import get_writer

## Constantes y variables auxiliares

In [2]:
ESTADOS = {1:"Aguascalientes", 2:"Baja California", 3:"Baja California Sur", 4:"Campeche", 
            5:"Chiapas", 6:"Chihuahua", 7:"Coahuila", 8:"Colima", 9:"Durango", 10:"Guanajuato",
            11:"Guerrero", 12:"Hidalgo", 13:"Jalisco", 14:"Ciudad de México", 15:"Estado de México", 
            16:"Michoacán", 17:"Morelos", 18:"Nayarit", 19:"Nuevo León", 20:"Oaxaca", 21:"Puebla", 22:"Querétaro", 
            23:"Quintana Roo", 24:"San Luis Potosí", 25:"Sinaloa", 26:"Sonora", 27:"Tabasco", 28:"Tamaulipas",
            29:"Tlaxcala", 30:"Veracruz", 31:"Yucatán", 32:"Zacatecas"}

# Modidificar si deseas guardar los audios o captions en otra ubicación
AUDIOS_DIR = os.path.join(os.getcwd(), "audios")
CAPTIONS_DIR = os.path.join(os.getcwd(), "captions")
YT_CAPTIONS_DIR = os.path.join(CAPTIONS_DIR, "yt_captions")
WHISPER_CAPTIONS_DIR = os.path.join(CAPTIONS_DIR, "whisper_captions")

## Creando un dataframe del corpus actual

Actualizado el: 11/11/2023

In [3]:
df = pd.read_csv("corpus_yt_2023.csv")
df.rename(str.strip, axis="columns", inplace = True)
df.loc[:, "Nombre"].ffill(limit=2, inplace = True)
df.loc[:, "Estado"].ffill(inplace=True)
df.loc[:, "Edad"].ffill(limit=2, inplace = True)
df.loc[:, "Género"].ffill(limit=2, inplace = True)

df.head(5)

,Estado,Nombre,Edad,Género,Título,Fecha de creación,Link,Fecha de recopilación,Aportación
0,Aguascalientes,Alejandra García,26.0,Femenino,REGALO COMÍDA en la CALLE | lo logramos 🙏🏻🥳 Al...,31 agosto 2023,https://youtu.be/hWKy7Ns1pNo?si=CtYGM3G0f_TtUXUz,18 septiembre 2023,Heili
1,Aguascalientes,Alejandra García,26.0,Femenino,"habitación principal | REMODELACIÓN, CON POQUI...",30 noviembre 2022,https://youtu.be/bm2xz0_HrGc?si=anegbcnciRTpv535,18 septiembre 2023,Heili
2,Aguascalientes,Alejandra García,26.0,Femenino,Nuestro Árbol de Navidad Casero | Ale García ✨,21 noviembre 2021,https://youtu.be/JT3WoDrrl1A?si=U7yvebArA8Qx6wkv,18 septiembre 2023,Heili
3,Aguascalientes,Paulina Espinoza Ávila,37.0,Femenino,inflamación abdominal,1 agosto 2023,https://youtu.be/6DRWV94GF_M?si=-bhaBjaK_jonBLNz,26 de octubre 2023,Heili
4,Aguascalientes,Paulina Espinoza Ávila,37.0,Femenino,6 de enero de 2023,6 enero 2023,https://youtu.be/8rAqZKE6ZRk?si=gH_-kllispldo99U,26 de octubre 2023,Heili


## Database con TinyDB

In [5]:
db_file = "audio_database.json"
DOWNLOADED = [] # Guarda el número de la fila de los audios descargados

# Tratamos de abrir el archivo json, en caso de que no este, lo creamos.
try:
    with open(db_file, 'r'):
        pass
    db = TinyDB(db_file)
except FileNotFoundError:
    db = TinyDB(db_file)


def save_audio_index(index_downloaded):
    with open("downloaded.txt", 'a') as file:
        file.write(str(index_downloaded) + '\n')

# Guardamos: video_id, estado, autor, edad, genero, ruta_audio, yt_trad, whisper_trad
def create_db_entries(df_entry, r_aud, r_yt, r_whisp):
    id = YouTube(df_entry["Link"]).video_id
    db.insert({"Estado":df_entry["Estado"], "Autor":df_entry["Nombre"], "Edad":df_entry["Edad"], 
               "Genero":df_entry["Género"], "VideoID":id, "Ruta":r_aud, "Caption_YT": r_yt,
               "Caption_Whisper": r_whisp})

def load_downloaded_audios():
    # archivo txt con los indices de los auidos ya descargados
    try:
        with open("downloaded.txt", "r") as file:
            for row_num in file:
                val = int(row_num.strip("\n"))
                DOWNLOADED.append(val)
    except FileNotFoundError as e:
        print(f"File not found: {e}")

def errase_db():
    db.truncate()
    db.all()

def errase_downloaded_txt():
    try:
        with open("downloaded.txt", "w") as _:
           pass
    except FileNotFoundError as e:
        print(f'File not found: {e}')

## Funciones para generar las transcripciones. 

Las transcripcciones, o subtítulos o _captions_ es el texto generadoro en cada video. En este caso, utilizamos transcripciones generadas por dos métodos: 

1. Generadas de manera automática por YouTube.
2. Generadas por medio de Whisper.

In [25]:
# Generación de transcripciones
def captions_whisper(ruta_audio: str) -> str:
    model = whisper.load_model("base")
    audio = r'{}'.format(ruta_audio)
    result = model.transcribe(audio=audio,
                            fp16=False, word_timestamps=True,
                            verbose=False)
    srt_writer = get_writer("srt", WHISPER_CAPTIONS_DIR)
    srt_writer(result, audio)  

    n = ruta_audio.find('\\')
    name = ruta_audio[n+1:len(s)-4] + ".srt"
    ruta = WHISPER_CAPTIONS_DIR + '\\' + name

    return ruta

def captions_youtbe(yt: YouTube) -> str:
    vid_id = yt.video_id
    flag = True
    try: 
        caption = yt.captions.get_by_language_code("a.es")
        file_path = os.path.join(YT_CAPTIONS_DIR, f"{vid_id}.txt")
        caption.save_captions(file_path)
    except:
        print(f"No se pudo descargar los subtitulos del video: {vid_id}")
        flag = False

    ruta = file_path if flag else "N/A"

    return ruta
    

## Función para descargar videos

In [6]:
def download_audio(link, row, i):
    
    try: 
        yt = YouTube(link, on_progress_callback = on_progress)
    except VideoUnavailable:
        print(f'Video at {link} is unavailable, skipping...')
    else:
        try:
            ys = yt.streams.get_audio_only()
            try:
                current_name = ys.download(output_path=AUDIOS_DIR, mp3=True)
            except:
                # No se pudo guardar como mp3
                current_name = ys.download(output_path=AUDIOS_DIR)

            _, ext = os.path.splitext(current_name)  # Obtenemos la extension
            dest_path = os.path.join(AUDIOS_DIR, f"{yt.video_id}{ext}")
            os.rename(current_name, dest_path) # Cambiamos el nombre por su id
            #save_audio_index(i) 
            # Funcion para generar las transcripciones
            rYT, rWHIS = generate_transcriptions(yt)
            create_db_entries(row, dest_path, rYT, rWHIS)
        except Exception as e:
            print(f'An error occurred while downloading the video: {str(e)}')

## Main

In [7]:
downloaded_counter = 0 

df_available = df.dropna()
available_total = len(df_available)
load_downloaded_audios()
exp_time = lambda x: x*71.8/5/60
print('**'*30)
print(f'Iniciando descarga de {available_total} audios.\nTiempo esperado de descarga: {round(exp_time(available_total),2)} minutos')

start_time = time.time()
for i, row in df_available.iterrows():

    link = row["Link"]

    if downloaded_counter % 5 == 0 and downloaded_counter != 0: 
        # Cada 5 descargas esperamos un minuto para evitar ser bloqueados
        time.sleep(60)

    if i in DOWNLOADED:
        print("Audio ya descargado, saltando...")
        continue
    else:
        #download_audio(link, row, i)
        pass

    downloaded_counter += 1

end_time = time.time()
total_time = round(end_time - start_time / 60, 2)
print("DESCARGA FINALIZADA")
print(f'Tiempo real de descarga: {total_time} minutos')
time.sleep(5)

### Creacion de captions


************************************************************
Iniciando descarga de 183 audios.
Tiempo esperado de descarga: 43.8 minutos


## TO DO 

- Agregar captions
    - Whisper
    - YouTube
- Actualizar la creacion de tinyDB con rutas de captions
- Terminar el querying 
- Arreglar archivos, formato, etc

In [9]:
#### delete
errase_db()
errase_downloaded_txt()